- 机器学习：
    - 通过让机器学习的方式，来达到某种实现功能的过程
    - 训练样本 + 特征 + 分类器
    - 机器学习需要明确的特征提取
    
- 深度学习：
    - 海量训练样本 + 神经网络
        - 在神经网络搭建的时候，框架自己训练特征，人不知道训练的是哪一个或多个特征

- 特征：
    - Haar特征：人脸识别
    - Hog特征：行人检测，车辆检测，物体检测

- 分类器：
    - 特征提取完之后，如何进行判决
        - adaboost分类器
        - svm分类器
        
- 如果要识别一个目标，需要**样本，特征，分类器**
    - 样本的准备
    - 特征的计算
    - 分类器的训练
    - 预测（检验）

- 如何进行样本收集
    - 通过视频获取样本数据
    
- Haar + adaboost ==> 人脸识别
- Hog + svm ==> 行人识别，车辆识别，物体检测

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 1. 视频分解图片

- 如何进行样本收集
    - 通过视频获取样本数据

In [2]:
cap = cv2.VideoCapture("./Wow Thing.mp4")    # 获取一个视频打开的句柄

isOpened = cap.isOpened    # 判断是否打开
print(isOpened)

fps = cap.get(cv2.CAP_PROP_FPS)                    # 获取帧率
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))    # 获取高
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))      # 获取宽

print(fps, height, width)
            
i = 0
while(isOpened):
    if i == 10:
        break
    else:
        i+=1
    flag,frame = cap.read()    # 读取每一张图。返回1：是否成功读取，返回2：图像数据
    fileName = "./image"+str(i)+".jpg"
    print(flag, fileName)
    if flag == True:
        cv2.imwrite(fileName,frame,[cv2.IMWRITE_JPEG_QUALITY, 100])  # 参数1：写入图片的名字，参数2：写入图片的数据，参数3：图像的质量控制

print("end!")

<built-in method isOpened of cv2.VideoCapture object at 0x10ee294f0>
29.97 1080 1920
True ./image1.jpg
True ./image2.jpg
True ./image3.jpg
True ./image4.jpg
True ./image5.jpg
True ./image6.jpg
True ./image7.jpg
True ./image8.jpg
True ./image9.jpg
True ./image10.jpg
end!


# 1.2 图片合成视频

- 创建写入的视频对象API：
    - cv2.VideoWriter()
        - 参数1：文件名称
        - 参数2：-1 选择一个可使用的解码器
        - 参数3：5  帧率，每秒5张
        - 参数4：size 大小




In [4]:
img = cv2.imread("./image1.jpg")
size = (img.shape[0],img.shape[1])
print(size)

# 创建写入对象
videoWrite = cv2.VideoWriter("2.mp4",-1,5,size)

# 把image1.jpg ~ image10.jpg的图片转换成视频
for i in range(1,11):
    fileName = "image" + str(i) + ".jpg"
    img = cv2.imread(fileName)
    videoWrite.write(img)    # 把编码之前的图片数据，写入视频，参数：图片

print("end!")

(1080, 1920)
end!


# 2. 图像的特征：

- 特征：
    - 图像中的某个像素点，经过某种四则运算后得到的结果（某一个区域内，像素点运算之后的结果）
        - 结果可能是
            - 具体的值
            - 向量、矩阵、多维

- 判决：
    - 利用特征区分目标：阈值判决
    - 多级目标的阈值判决
        - haar > T1 and haar > T2 ……
 
- 得到判决：
    - 通过机器学习得知判决门线，根据计算的特征进行阈值判决
    
    
    

## 2.1 Haar特征

- 种类：
    - 边缘特征：4种
    - 线性特征：8种
    - 圆形环绕特征：2种
    - 特点方向的特征：1种
    - 合计15种模板


- 公式：
    - 整体特征 1， 黑色特征-2， 白色特征3
    - 特征 = 某个区域*整体权重 + 黑色区域*(黑色特征) = (白色区域+黑色区域)*1 + 黑色区域 * -2
    - = 白色区域 + 黑色区域 - 2*黑色区域
    - = 白色区域 - 黑色区域
    
- 遍历：
    - 从上到下，从左到右，依次遍历
    - 例：
        - 图片尺寸1080x720，原始模板尺寸10x10，步长2
        - 特征计算量 = 15(种模板) * 20 (级缩放) *  (1080/2 x 720/2) * 100(10x10模板的四则运算) = 50-100亿
        - 实时处理图片：15帧每秒
            - 特征计算量 = 15 * (50-100)亿 = 1000亿次
            
- 积分图：
    - 任意的一个方块，都能由相邻的abcd运算得到
    - 公式：
        - 特征 = (p1-p2-p3+p4) * w
        
        
- 利用特征，区分目标，进行阈值判决
    - haar特征，一般搭配adaboost分类器，进行人脸识别
    - adaboost分类器能把每次的错误样本的**系数加强**，再把检测结果送到下一个检测器中，循环
        - 训练1：苹果0.1 苹果0.1 苹果0.1 香蕉0.5
            - 有余香蕉是不需要的物体，所以系数就比较大
    - 循环(训练)的终止条件
        - for循环如果大于某一个次数，迭代终止
        - 设定一个误差概率，若小于，则终止
            - 每次迭代完后，都会有一个检测概率。
            - 训练1：苹果0.1 苹果0.1 苹果0.1 香蕉0.5， 此时的正确概率为75%




## 2.2 Adaboost分类器的结构(3层结构，从下往上)


 - 一个【adaboost分类器】由若干个【强分类器】组成：
    - 一个强分类器【x1】= sum(y1,y2,y3)
    
    - 强分类器：判断当前阈值与特征值知否吻合，来达到目标【判决】的效果，通常使用多级目标的阈值判决
        - 第1个强分类器：判断特征值【x1】与门限阈值【t1】
        - 第2个强分类器：判断特征值【x2】与门限阈值【t2】
        - 第3个强分类器：判断特征值【x3】与门限阈值【t3】
        - 判断：
            - x1>t1 and x2>t2 and x3>t3
            - 若都通过，则目标为苹果
            - 若任意一个不通过，则目标为非苹果


- 一个【强分类器】由若干个【弱分类器】组成
    
    - 一个弱分类器的特征值【y1】= sum(z1,z2,z3)
    
    - 【y1】与对应的门限阈值【t1】比较
        - 若大于，则y1 = AA；反之，y1 = BB
        
    - 依次计算y2，y3……，共同构成一个强分类器的特征值【x1】
            
    - 弱分类器用于计算一个强分类器的特征值：x1，x2，x3


- 一个【弱分类器】由若干个【特征节点(node)】组成

    - 在OpenCV中，一个弱分类器，最高支持3个haar特征。每一个haar特征，构成一个**特征节点**
    
    - node1对应的特征值【z1】与node1对应的门限阈值【t1】比较
        - z1:经过计算得到的特征值
        - 若大于，则z1=alpha1；反之，z1=alpha2

    - 依次计算z2，z3……，共同构成一个弱分类器的特征值【y1】
            
    - 特征节点用于计算一个弱分类器的特征值：y1，y2，y3
    
    
## 2.3 Adaboost训练

- 步骤：
    1. 初始化数据的权重分布：
        - 苹果：0.1，苹果：0.1，苹果：0.1，香蕉：0.1
    2. 遍历判决阈值P
        - 会计算出一系列的误差概率
        - 选取最小的误差概率：minP，和对应的阈值t
    3. 计算当前的权重系数G1(x)
    4. 更新权重分布
        - 苹果：0.2，苹果：0.2，苹果：0.2，香蕉：0.8
    5. 如果训练没完成，返回第2步，把更新后的权重分布作为训练样本，重新遍历阈值，计算误差概率，计算权重G2(x)、G3(x)、G4(x)……
 

## 人脸检测（haar + adaboost）

In [15]:
# 1. 导入2个xml文件（1个描述人脸，1个描述眼睛）和jpg图片
# 2. gray jpg，haar基于灰度图片进行计算
# 3. 计算haar特征（cv2已经做了）
# 4. 检测当前haar特征的人脸，以及人脸上的眼睛
# 5. 对检测的结果进行遍历，绘制检测人脸和人眼的方框

# 2个xml：已经训练好的人脸、人眼识别的adaboost分类器

face_xml = cv2.CascadeClassifier("./haarcascade_frontalface_default.xml")
eye_xml = cv2.CascadeClassifier("./haarcascade_eye.xml")

image = cv2.imread("./2ppl.jpg",1)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("src", image)

# 人脸检测
faces = face_xml.detectMultiScale(gray, 1.2, 5)
# 参数1：灰度图片数据
# 参数2：1.3 模板的缩放系数scale
# 参数3：5 人脸最小不能小于5个像素

print("face=", len(faces))    # 检测当前的人脸数

# 给人脸画方框，(x,y) 人脸检测的起始坐标
for (x,y,w,h) in faces:
    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 2)
    roi_face = gray[y:y+h, x:x+w]    # 定位人脸区域，检测人眼
    roi_color = image[y:y+h, x:x+w]
    eyes = eye_xml.detectMultiScale(roi_face, 1.2, 5) # 必须灰度图像
    print("eye=", len(eyes))
    for (e_x,e_y,e_w,e_h) in eyes:
        cv2.rectangle(roi_color, (e_x, e_y), (e_x+e_w , e_y+e_h), (0,255,0),2)


cv2.imshow("dst", image)
cv2.waitKey(5000)
cv2.destroyAllWindows()

face= 2
eye= 0
eye= 3


In [ ]:
# 使用摄像头，人脸检测

capture = cv2.VideoCapture(0)
while True:
    flag, image = capture.read()
    image = cv2.flip(image, 1)

    eye_xml = cv2.CascadeClassifier("/Users/zhengtaizhong/Desktop/T+O/haarcascade_eye.xml")
    face_xml = cv2.CascadeClassifier("/Users/zhengtaizhong/Desktop/T+O/haarcascade_frontalface_default.xml")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = face_xml.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi_face = gray[y:y + h, x:x + w]  # 定位人脸区域，检测人眼
        roi_color = image[y:y + h, x:x + w]
        eyes = eye_xml.detectMultiScale(roi_face, 1.3, 5)  # 必须灰度图像

        for (e_x, e_y, e_w, e_h) in eyes:
            cv2.rectangle(roi_color, (e_x, e_y), (e_x + e_w, e_y + e_h), (0, 255, 0), 2)

    
    cv2.imshow("face", image)
    cv2.waitKey(50)



# cv2.destroyAllWindows()

# Hog特征 + SVM分类器

## SVM分类器

- 分类：
    - 将数据集分割开来的直线，称为分割超平面，也就是分类的决策边界
        - 若数据点都在二维平面上，则分割超平面就是一条【直线】
        - 若数据集是三维，则分割超平面就是一个【平面】
        - 若数据集是n维，则分割超平面就是n-1维
        
- 三维分类：
    - 将二维的点，投影到三维空间，使用一个超平面进行数据切割（分类）
    - 寻求最优超平面，完成分类

- 核：
    - 线性核

- 数据：
    - -1类和1类
    - 每类数量可不相等
    - 每组数据需要一个相应的目标值（描述当前数据唯一的一组属性）
    
- 步骤：
    - 实例化svm，训练，预测
    
    
- 监督学习
    - 案例：身高体重分类


In [14]:
girls = np.array([[155,48],[159,50],[164,53],[169,56],[172,60]])
boys = np.array([[152,53],[156,55],[160,56],[172,64],[176,65]])

# 准备label（标签值、目标值）：
# svm 所有的数据都要有label（标签值、目标值），描述当前数据唯一的一组属性
label = np.array([[-1],[-1],[-1],[-1],[-1],[1],[1],[1],[1],[1]])      # 表示当前数据的目标值，-1是女生，1是男生


# 合并男女生数据，并转换成浮点型
data = np.vstack((girls, boys))
data = np.array(data, dtype="float32")

# 创建一个支持向量机的对象
svm = cv2.ml.SVM_create()

# 设置属性
svm.setType(cv2.ml.SVM_C_SVC)    # 设置SVM的类型
svm.setKernel(cv2.ml.SVM_LINEAR)    # 设置SVM的内核属性：由于使用线性分类器，所以是线性核
svm.setC(0.01)    # 与svm核有关的参数

# 训练
result = svm.train(data,cv2.ml.ROW_SAMPLE, label)   # 特征值，训练类型，目标值

# 【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】【】
# print(result.rho)
alpha = np.zeros((1), np.float32)
rho = svm.getDecisionFunction(0, alpha)
print(rho)

# 预测（使用predict检测）
pt_data = np.vstack([[167,55],[162,57]])    # 第一个0（女生），第二个1（男生）
pt_data = np.array(pt_data, dtype="float32")
print(pt_data)
par1, par2 = svm.predict(pt_data)
print(par2)

(1.472660550458933, array([[1.]]), array([[0]], dtype=int32))
[[167.  55.]
 [162.  57.]]
[[-1.]
 [ 1.]]


# Hog 特征

- 特征：某个像素，经过某种运算得到的结果

- haar 特征：通过模板计算，得到的特征值

- Hog的计算步骤：
    - 特征的模块划分
        - 模块(整张图) > 检测窗口window  > block > cell
        - 每层都有size。每层都紧密相连，只有第一层（window）的size定义好之后，后面的层，依次参考前面层的size
        - block与window在滑动时，都有步长
        - cell bin，无步长
        
        - window：特征计算最顶层单元
            - window在image里，从上到下，从左到右依次滑动，
            - 包含一个目标（object）所有的特征，根据这个特征，对目标进行判决
            - size：128x64(官方：人脸识别，车辆检测）
            
        - block
            - block在window里，从上到下，从左到右依次滑动，
            - size：window的宽高，是block的整数倍。一般是16x16
            - 步长：描述block如何在window中，进行滑动。一般是8x8（可不一样），水平方向滑动步长为8，竖直方向滑动步长为8
            - 一个window中block的数量 = ((128-16)/8+1) * ((64-16)/8+1) = 105
                - 第一个8，沿竖直方向上滑动的步长；第一个16，block的高度
                - 第二个8，沿水平方向上滑动的步长；第二个16，block的宽度
        - cell
            - size: 8x8
            - cell是block的元素，一个block中，包含(16x16)/(8x8) = 4个cell，无法滑动
            - cell1, cell2, cell3, cell4
            - bin：
                - 每个像素，都有梯度。梯度包含大小(f)和方向(angle)，方向的范围为[0,360]
                - 按40°一块，划分360°。一共有9块，每一块就是一个bin。所以一个bin等于40°，360°有9个bin
                - 40°非连续，每20°呈180°对称关系：
                    - bin1 = [0,20] + [180,200]
                    - 在这个夹角范围内的，都在一个bin里
                - 每个cell里面有很多元素，这些元素共同描述了360度，总共9个bin的信息。每个block4个cell，36个bin
                
                
        - hog特征维度：
            - haar特征值是一个值，hog特征值是一个向量(完全描述物体的特征)
            - hog维度 = 每个window中block的数量(105) x 每个block中cell的数量(4) x 每个cell中bin的数量(9) = 3780
            
    - 根据模板，计算梯度和方向
    - 根据梯度和方向，进行bin的投影
    - 计算每个模块的hug特征
    
    - 梯度的方向和大小
        - 每个像素都有一个梯度，检测窗口的所有像素组成在一起，才构成hog特征。
        - 在图像中梯度的概念也是像素值变换最快的方向
        - 特征模板：
            - x方向的梯度a: 竖直方向的像素 x [1,0,-1]，
            - y方向的梯度b: 水平方向的像素 x [[1],[0],[-1]]
            - 某像素与两个模板，做卷积
            - 梯度f = sqrt(a^2 + b^2)
            - angle = arctan(a/b)
            
    - 小结：
        - 1个cell = 360° = 9个bin
        - 1个block中有4x9=36个bin的向量
        - 1个window有105个block，共有105x36=3780个bin的向量
    
    
    - bin的投影与投影的幅度：
        - 假设一个像素点x，梯度f，角度a，bin1 = [0,20] + [180,200]
            - 若a = 10°：则x位于[0,20]的中间，x投影在bin1上，x投影的幅度就是f
        - bin的分解：分解到相邻的两个bin的范围内：bin1、bin2
            - 若a = 25°，
                - 分解在bin1的幅度f1 = 梯度f * f(角度)
                - 分解在bin2的幅度f2 = 梯度f * (1-f(角度))
                    - f(角度）在[0,1] 
                    
                    
   - 计算整体hog特征
       - 3780维的检测窗口，包含block cell bin，每一个维度，就是一个bin
           - 在1个block里，cell0~3，bin0~8
       
       - 像素值233投影在某1个block的cell0中，经过特征模板计算梯度，得到该像素方向angle和幅值f0
           - angle决定该像素投影到某个bin上，假设投影在bin0，则bin0=f0
           - 假设另一个像素值也投影在bin0上，则该像素投影在bin0上的值为f1
           - 权重累加：
               - sum_bin0 = f0+f1+……
               - 还要加上当前像素在其他像素的投影，即bin1
        
       - cell 复用
           - cell0、cell1……cell8
           - cellx0、cellx2、cellx4
             - 假设像素[i,j]投影到cell0的bin0+bin1
                 - cellx0，当前像素只能投影在cell0的bin中
                     - 只对当前cell起作用
                 
                 - cellx2，当前像素能同时投影在cell0和cell1的2个bin中，2个cell能分出4个bin
                 
                 - cellx4，当前像素能同时投影在4个cell的相同的2个bin中。4个cell能分出8个bin
                     - 对cell0~cell3都起作用
       - 判决门线：
           - hog*svm = 值，
           - 这个值就与判决门限进行比较，
               - 若大于，则是目标；否则，非目标

# 案例：物体识别：

1. 样本
    - 一个好的样本远胜过一个复杂的神经网络
    
    - 正负样本比例：1:2 ~ 1:3之间。按一定的规律命名
    
    - 正样本pos：包含所检测的目标,128x64
        - 尽可能多样化，
            - 环境多样化
            - 尽可能多的干扰
          
    - 负样本neg：不包含所检测的目标
    
    
    - 样本获取(样本的来源)：
        - 网络公司 卖样本
        - 爬虫
        - 公司内部的积累
        - 自己收集：视频，100秒的视频，30fps，一共3000张
        
    - 样本准备
        - 图片的缩放和裁减(图片的几何变化)
2. hog+svm 训练
3. 检验：找一张图片进行预测


## 代码实现步骤：

1. 参数的设置
2. 创建一个hog实例对象
3. 计算hog
4. label标签
5. 创建svm，并设置属性
6. 完成训练
7. 完成预测
8. 绘图

In [34]:
#1.参数的设置
PosNum = 820           # 正样本的个数
NegNum = 1931          # 负样本的个数
winSize = (64,128)     # 检测窗口的大小
blockSize = (16,16)    # block大小
blockStride = (8,8)    # block的步长
cellSize = (8,8)       # cell的大小
nBin = 9               # bin的个数


# 2. 实例化hog对象，hog describe获取hog特征的维度
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nBin)


# 3. 计算所有正负样本的hog值:
featureNum = int(((128-16)/8+1)*((64-16)/8+1)*4*9)                    # 特征维度：3078
featureArray = np.zeros(((PosNum+NegNum), featureNum), np.float32)    # 初始化特征值
labelArray = np.zeros(((PosNum+NegNum), 1), np.int32)                 # 初始化目标值

# 遍历所有正样本图片
for i in range(0, PosNum):
    fileName = './pos/'+str(i+1)+'.jpg'          # 获取图片名
    img = cv2.imread(fileName)                   # 加载图片数据
    hist = hog.compute(img, (8,8))               # 当前hog的特征计算，维度：3780维
    for j in range(0, featureNum):
        featureArray[i,j] = hist[i]              # 把hog的计算的特征值，装入特征值数组中
    labelArray[i,0] = 1                          # 添加每张图片的目标值 1 
    
# 遍历所有负样本图片
for i in range(0, NegNum):
    fileName = './neg/'+str(i+1)+'.jpg'          # 获取图片名
    img = cv2.imread(fileName)                   # 加载图片数据
    hist = hog.compute(img, (8,8))               # 当前hog的特征计算，维度：3780维
    for j in range(0, featureNum):
        featureArray[i+PosNum,j] = hist[i]       # 把hog的计算的特征值，装入特征值数组中
    labelArray[i+PosNum,0] = -1                  # 添加每张图片的目标值-1

    
# 5. 实例化svm对象
svm = cv2.ml.SVM_create()

# svm属性设置
svm.setType(cv2.ml.SVM_C_SVC)       # svm类型
svm.setKernel(cv2.ml.SVM_LINEAR)    # kernel类型
svm.setC(0.01)                      # kernel参数


# 6. 训练
ret = svm.train(featureArray, cv2,ml.ROW_SAMPLE, labelArray)    # 把样本训练好之后，得到【sv】

# 7. 预测（使用detectMultiScale检测，而非predict）
# 核心：创建【myHog】，为了设置【myDetect】参数，
# 【myDetect】本质是一个array，包含【resultArray】和【rho】
# 【resultArray】通过公式：resultArray = -1 * alphaArray * svArray计算
# 【rho】由svm训练得到
# 得到【myHog】后，使用【detectMultiScale】方法完成检测

alpha = np.zeros((1), np.float32)            # [0.]
rho = svm.getDecisionFunction(0, alpha)      # 通过【svm】，得到的hog描述信息【rho】，rho1行1列的1维数组
print(rho) 
print(alpha)


alphaArray = np.zeros((1,1),np.float32)                # [[0.]]
svArray = np.zeros((1,featureNum),np.float32)          # 支持向量的个数： (1,3780)
resultArray = np.zeros ((1,featureNum),np.float32)     # (1,3780)
resultArray = -1 * alphaArray * svArray                # -1x(1,1)x(1,3780)=[[-0., -0., -0., ..., -0., -0., -0.]]
alphaArray[0,0] = alpha

                   
# 设置【myDetect】参数包括【resultArray】和【rho】
myDetect = np.zeros ((3781),np.float32) 
for i in range(0,3780):                     # 遍历3780维里（列），每一个元素
    myDetect[i] = resultArray[0,i]          # myDetect的前3780维来自于resultArray
myDetect[3780] = rho[0]                     # 最后的判决需要用到【rho】。myDetect的第3781维来自于rho


# 创建一个hog，传入myDetect参数
myHog = cv2.HOGDescriptor() 
myHog.setSVMDetector(myDetect)    # myHog中已包含被训练出来的结果


# 加载目标图片，并检测
imageSrc = cv2.imread('Test2.jpg',1)
# 对预测样本(目标图片)的检测，(8,8)win的滑动步长，(32,32)win大小，1.05缩放系数
objs = myHog.detectMultiScale(imageSrc,0,(8,8),(32,32), 1.05,2)
# objs三维，返回的x,y,w,h在第三维度
x = int(objs[0][0][0])    
y = int(objs[0][0][1])
w = int(objs[0][0][2])
h = int(objs[0][0][3])


# 预测样本的绘制与展示
cv2.rectangle(imageSrc, (x,y),(x+w,y+h),(255,0,0),2)
cv2.imshow("dst",imageSrc)
cv2.waitKey(5000)
cv2.destroyAllWindows()

error: OpenCV(3.4.2) /opt/concourse/worker/volumes/live/9523d527-1b9e-48e0-7ed0-a36adde286f0/volume/opencv-suite_1535558719691/work/modules/core/src/matrix.cpp:755: error: (-215:Assertion failed) dims <= 2 && step[0] > 0 in function 'locateROI'
